In [1]:
# This notebook deals with creating an edgelist file containing all the edges in the nth largest
# connected component of the large edgelist (600 million edges). It does not rely on NetworkX
# or any other libraries other than Pandas to do this (since they are too computationally expensive)
import pandas as pd
import time

In [2]:
class UnionFind:
    def __init__(self, n):
        self.parent = [i for i in range(n)]
        self.rank = [0] * n
        self.size = [1] * n

    def find(self, u):
        if self.parent[u] != u:
            self.parent[u] = self.find(self.parent[u])
        return self.parent[u]

    def union(self, u, v):
        pu, pv = self.find(u), self.find(v)
        if pu == pv:
            return
        if self.rank[pu] < self.rank[pv]:
            self.parent[pu] = pv
            self.size[pv] += self.size[pu]
        elif self.rank[pv] < self.rank[pu]:
            self.parent[pv] = pu
            self.size[pu] += self.size[pv]
        else:
            self.parent[pu] = pv
            self.rank[pv] += 1
            self.size[pv] += self.size[pu]

In [38]:
def initialize_unionfind(edges):
    print("Initializing UnionFind data structure ...")
    nodes = set(edges['source']).union(set(edges['target']))
    node_index = {node: i for i, node in enumerate(nodes)}
    n = len(nodes)
    uf = UnionFind(n)
    print("Computing union of all edges ...")
    for _, row in edges.iterrows():
        uf.union(node_index[row['source']], node_index[row['target']])
    print("Done.")
    return uf, node_index

In [71]:
def get_connected_component(uf, node_index, n=0):
    comps = {}
    for node, i in node_index.items():
        parent = uf.find(i)
        comps[parent] = comps.get(parent, []) + [node]
    if n >= len(comps):
        return []
    keys_sorted = sorted( comps.keys(), reverse=True, key=lambda parent: len(comps[parent]) )
    return comps[keys_sorted[n]]

In [61]:
def get_amount_of_components(uf, node_index):
    # Count unique parents
    unique_parents = set()
    for i in range(len(node_index)):
        unique_parents.add(uf.find(i))
    return len(unique_parents)

In [4]:
# Variables
edges_fn = "data/edges.csv"
edges_total = 684_732_453 # hardcoded

In [25]:
# Read edge list to df
perc = 1
nrows=int(edges_total*perc/100)
df = pd.read_csv(edges_fn, nrows=nrows)
print("read {:_} lines".format(len(df)))

reading edges ... read 6_847_324 lines (took 2.1s)


In [41]:
# Initialize union find
uf, node_index = initialize_unionfind(df)

Initializing UnionFind data structure ...
Computing union of all edges ...
Done.


In [72]:
# Get nth largest connected component
for i in range(get_amount_of_components(uf, node_index)):
    component = get_connected_component(uf, node_index, i)
    print("Nodes in the {} largest connected component: {}".format(i+1, len(component) if component != None else None))

Nodes in the 1 largest connected component: 12433
Nodes in the 2 largest connected component: 221
Nodes in the 3 largest connected component: 136
Nodes in the 4 largest connected component: 36
Nodes in the 5 largest connected component: 32
Nodes in the 6 largest connected component: 18
Nodes in the 7 largest connected component: 5


In [65]:
print(get_amount_of_components(uf, node_index))

7


In [35]:
# Get edgelist of connected component
dfcc = df[ (df['source'].isin(component)) & (df['target'].isin(component)) ]
print("{:_} / {:_}".format(len(dfcc), len(df)))

6_814_841 / 6_847_324


In [45]:
# Save new edges to file
dfcc.to_csv("data/edges_cc_test.csv", encoding='utf-8', index=False)